In [45]:
import numpy as np
import torch
import opt_einsum
import itertools

import process_matrices
import utils

from tqdm import tqdm

torch.cuda.empty_cache()

In [46]:
q_s = 1
q_c = 4

In [47]:
# Generalized Amplitude Damping Krauss Operators
N = 0
g = 0.5

K_1 = torch.tensor([[np.sqrt(1 - N), 0],[0, np.sqrt(1 - N) * np.sqrt(1 - g)]])
K_2 = torch.tensor([[0,np.sqrt(g*(1-N))],[0,0]])
K_3 = torch.tensor([[np.sqrt(N)*np.sqrt(1-g), 0],[0,np.sqrt(N)]])
K_4 = torch.tensor([[0,0],[np.sqrt(g * N), 0]])
K = [K_1, K_2, K_3, K_4]

In [48]:
pms = process_matrices.ProcessMatrices(q_s, q_c, K=K, device="cpu")

In [49]:
X_C, X_E, X_R = pms()

In [50]:

optimizer_C = torch.optim.SGD([pms.X_C], lr = 1e-3)
optimizer_R = torch.optim.SGD([pms.X_R], lr = 1e-3)


In [51]:
pbar = tqdm(range(1000))
regularization = 1
for epoch in pbar:
    for _ in range(1000):
        # X_C Optimizing
        optimizer_C.zero_grad()
        X_C, X_E, X_R = pms()
        
        f_avg = utils.avg_fidelity(X_C, X_E, X_R)
        
        l = -f_avg
        l.backward()
        optimizer_C.step()

        with torch.no_grad():
            pms.X_C.data = utils.make_PSD(pms.X_C.data)
            pms.X_C.data = utils.make_sum_to_identity(pms.X_C.data)

    for _ in range(1000):        
        # X_R Optimizing
        optimizer_R.zero_grad()
        X_C, X_E, X_R = pms()
        f_avg = utils.avg_fidelity(X_C, X_E, X_R)

        l = -f_avg
        l.backward()
        optimizer_R.step()

        with torch.no_grad():
            pms.X_R.data = utils.make_PSD(pms.X_R.data)
            pms.X_R.data = utils.make_sum_to_identity(pms.X_R.data)
    
    description = f"Avg Fidelity : {f_avg:.4f}"
    pbar.set_description(description)

Avg Fidelity : 0.0039:   1%|          | 7/1000 [01:08<2:41:03,  9.73s/it]


KeyboardInterrupt: 

In [23]:
X_C, X_E, X_R = pms()